## Основы Python, numPy, pandas на примере данных Auto.ru

<font color=darkblue>Импортируем нужные пакеты

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import date as Date
from numpy import nan as NaN

%matplotlib inline

<font color=darkblue>Файлы с входными/выходными данными

In [2]:
INPUT_FILE = u'../data/auto.ru.csv'
OUTPUT_FILE = u'../data/auto.ru_data1.csv'

<font color=darkblue>Загрузка данных из csv-файла в датафрейм

In [5]:
#df = pd.read_csv(u'../data/auto.ru_data.csv')
df = pd.read_csv(u'../data/auto.ru_data.csv')

<font color=darkblue>Вывод первых пяти строк датафрейма

In [6]:
df.head(5)

,Год выпуска,КПП,Привод,Пробег,Тип кузова,Состояние,Срок владения,Цвет,Руль,Владельцы,...,Модель,Город,Объем двигателя,Мощность двигателя,Двигатель,Цена,Дата,Количество дней,Просмотров всего,Просмотров в день
0,2012.0,механическая,передний,141000.0,внедорожник 5 дв.,Не требует ремонта,NaN,белый,Левый,2 владельца,...,ix35 I,Воронеж,2.0,150.0,Бензин,790000,2018-06-25,60,387.0,6.450000
1,2014.0,автоматическая,полный,80189.0,внедорожник 5 дв.,Не требует ремонта,NaN,белый,Левый,3 или более,...,X1 I (E84) Рестайлинг 20d,Москва,2.0,184.0,Дизель,1368000,2018-06-25,60,657.0,10.950000
2,2010.0,механическая,передний,143000.0,седан,Не требует ремонта,NaN,бежевый,Левый,3 или более,...,Logan I Рестайлинг,Воронеж,1.4,75.0,Бензин,320000,2018-06-25,60,470.0,7.833333
3,2009.0,механическая,передний,126000.0,седан,Не требует ремонта,NaN,жёлтый,Левый,3 или более,...,Nexia I Рестайлинг,Россошь,1.5,80.0,Бензин,150000,2018-06-25,60,438.0,7.300000
4,2011.0,механическая,передний,159000.0,хэтчбек 5 дв.,Не требует ремонта,NaN,красный,Левый,3 или более,...,C4 II,Воронеж,1.6,120.0,Бензин,410000,2018-06-25,60,442.0,7.366667


<font color=darkblue>Количество строк в датафрейме

In [7]:
len(df)

20237

<font color=darkblue>Функция для разбора названия на марку и модель

In [8]:
def get_mark_and_model(name):
    if name is NaN:
        mark = NaN
        model = NaN
    else:
        # Массив моделей с названиями из нескольких слов
        auto_marks = ["Great Wall", "Land Rover", "Alfa Romeo"]
            
        mark = NaN
        model = NaN
        
        for auto_marks in auto_marks:
            if name.startswith(auto_marks):
                mark = auto_marks
                model = name[len(mark):].strip()
        
        if mark is NaN and model is NaN:
            words = name.split()
            mark = words[0]
            model = " ".join(words[1:])
       
    return mark, model

<font color=darkblue>Использование лямбда-выражений на примере применения функции разбора названия для каждого элемента столбца "Название"

In [9]:
df["Марка"], df["Модель"] = zip(*df["Название"].apply(lambda x: get_mark_and_model(x)))

KeyError: 'Название'

<font color=darkblue>Вывод первых десяти строк определенного набора столбцов датафрейма, заданного массивом наименований

In [10]:
df[["Название", "Марка", "Модель"]].head(10)

KeyError: "['Название'] not in index"

<font color=darkblue>Получение перечня уникальных значений столбца "Марка"

In [11]:
df.Марка.unique()

array(['Hyundai', 'BMW', 'Renault', 'Daewoo', 'Citroen', 'Volkswagen',
       'Peugeot', 'Ford', 'Skoda', 'SsangYong', 'Dodge', 'Chevrolet',
       'Kia', 'LADA', 'Mercedes-Benz', 'Subaru', 'Lifan', 'Brilliance',
       'Mazda', 'Great Wall', 'Nissan', 'Audi', 'Opel', 'Suzuki',
       'Mitsubishi', 'Lincoln', 'Datsun', 'Honda', 'ГАЗ', 'Toyota',
       'Chery', 'Volvo', 'Porsche', 'Jeep', 'ЛуАЗ', 'Cadillac', 'УАЗ',
       'Vortex', 'Chrysler', 'Jaguar', 'Infiniti', 'ИЖ', 'SEAT', 'Geely',
       'Rover', 'ЗАЗ', 'Acura', 'Land Rover', 'GMC', 'Lexus', 'MINI',
       'JAC', 'BYD', 'Fiat', 'Hawtai', 'Pontiac', 'Alfa', 'Changan',
       'FAW', 'DongFeng', 'Asia', 'Москвич', 'Isuzu', 'Saab', 'Daihatsu',
       'Hummer', 'ТагАЗ', 'Smart', 'Haima', 'Bentley', 'Luxgen', 'Buick',
       'Genesis', 'Haval', 'Zotye', 'Ravon', 'Plymouth', 'Lamborghini',
       'Saturn', 'Foton', 'MG', 'Tesla', 'Maserati'], dtype=object)

<font color=darkblue>Функция для разбора поля "Адрес" на "Город" и остальное

In [12]:
def get_address(address):
    if address is NaN:       
        city = NaN
        street = NaN
    else:
        words = address.split()
        city = words[0]
        city = city.strip(" ,")
        street = " ".join(words[1:])
       
    return city, street

<font color=darkblue>Использование лямда-выражения

In [13]:
df["Город"], df["Полный адрес"] = zip(*df["Адрес"].apply(lambda x: get_address(x)))
df[["Адрес", "Город", "Полный адрес"]].head(10)

KeyError: 'Адрес'

<font color=darkblue>Функция для разбора поля "Тип двигателя" на "Объем", "Мощьность" и "Двигатель"
Пример использования регулярных выражений

In [14]:
def get_engine_type(engine):
    if engine is NaN:       
        volume = NaN
        hp = NaN
        engine_type = NaN
    else:        
        regex = re.search("(\d+\.\d+)\sл\s/\s(\d+)\sл\.с\.\s/\s([а-яА-яёЁ]+)", engine)
    
        if regex is not None:
            volume = float(regex.group(1))
            hp = int(regex.group(2))
            engine_type = regex.group(3)
        else:           
            volume = NaN
            hp = NaN
            engine_type = NaN
       
    return volume, hp, engine_type

<font color=darkblue>Тип двигателя.

In [15]:
df["Объем двигателя"], df["Мощность двигателя"], df["Двигатель"] = zip(*df["Тип двигателя"].apply(lambda x: get_engine_type(x)))
df[["Тип двигателя", "Объем двигателя", "Мощность двигателя", "Двигатель"]].head(10)

KeyError: 'Тип двигателя'

<font color=darkblue>Функция для получения значения цены авто с преведением к типу int

In [16]:
def get_price(price_text):
    if price_text is NaN:
        price = NaN
    else:
        regex = re.search("[от ]*([\d*\s*]+)", price_text)
        
        if regex is not None:
            price = regex.group(1)
            price = price.replace("\xa0", "")
            price = int(price)        
        else:
            price = NaN
    
    return price



<font color=darkblue>Цена.

In [17]:
df["Цена"] = df["Стоимость"].apply(lambda x: get_price(x))
df[["Стоимость", "Цена"]].head(10)

KeyError: 'Стоимость'

In [18]:
def get_month(month_text):
    months = {
        "января": 1,
        "февраля": 2,
        "марта": 3,
        "апреля": 4,
        "мая": 5,
        "июня": 6,
        "июля": 7,
        "августа": 8,
        "сентября": 9,
        "октября": 10,
        "ноября": 11,
        "декабря": 12
    }
    return months[month_text]


def get_date_and_days(date_text, current_date):
    if date_text is NaN:
        date = NaN
        days = NaN
    else:
        if date_text == "Новый":
            date = current_date
            days = 1
        else:
            regex = re.search("(\d+)\s*([а-яё]+)\s*(\d*)", date_text)
        
            if regex is not None:
                day = int(regex.group(1))
                month = get_month(regex.group(2))
                
                if regex.group(3) == "":
                    year = current_date.year
                else:
                    year = int(regex.group(3))
                
                date = Date(year, month, day)
                days = (current_date - date).days + 1
            else:
                date = NaN
                days = NaN
        
    return date, days

<font color=darkblue>Количество дней, которое висит объявление.

In [19]:
today = Date(2018, 8, 23)
df["Дата"], df["Количество дней"] = zip(*df["Дата создания"].apply(lambda x: get_date_and_days(x, today)))
df["День"], df["Месяц"], df["Год"] = zip(*df["Дата"].apply(lambda x: (x.day, x.month, x.year)))
df[["Дата создания", "Дата", "День", "Месяц", "Год", "Количество дней"]].head(10)

KeyError: 'Дата создания'

<font color=darkblue>Функция для получения количества присмотров объявления (всего и сегодня)

In [20]:
def get_views_count(views_text):
    if views_text is NaN:
        total_views = NaN
        today_views = NaN
    else:    
        regex = re.search("(\d+)\s*\((\d+)\sсегодня\)", views_text)
    
        if regex is not None:           
            total_views = int(regex.group(1))
            today_views = int(regex.group(2))
        else:
            total_views = NaN
            today_views = NaN
        
    return total_views, today_views

<font color=darkblue>Количество просмотров.

In [21]:
df["Просмотров всего"], df["Просмотров сегодня"] = zip(*df["Кол-во просмотров"].apply(lambda x: get_views_count(x)))
df[["Кол-во просмотров", "Просмотров всего", "Просмотров сегодня"]].head()

KeyError: 'Кол-во просмотров'

<font color=darkblue>Функция для получения значения среднего количества просмотров объявления в день

In [ ]:
def get_mean_of_views(row):
    if row["Просмотров всего"] is NaN or row["Количество дней"] is NaN:
        mean = NaN
    else:
        mean = row["Просмотров всего"] / row["Количество дней"]
    return mean

<font color=darkblue>Количество просмотров в день.

In [ ]:
df["Просмотров в день"] = df.apply(lambda x: get_mean_of_views(x), axis=1)
df[["Просмотров всего", "Количество дней", "Просмотров в день"]].head()

<font color=darkblue>Вывод всех наименований столбцов датафрейма

In [ ]:
df.columns

In [ ]:
df.head()

<font color=darkblue>Удаление строк с пустыми значениями в определенных столбцах.

In [ ]:
count_before = len(df)
df = df[df.isnull().apply(lambda x: not(x["Название"] or x["Адрес"] or x["Тип двигателя"] or 
                                        x["Стоимость"] or x["Дата создания"] or x["Кол-во просмотров"]), axis=1)]
count_after = len(df)
print("Удалено {0} строк, осталось {1} строк.".format(count_before - count_after, count_after))

<font color=darkblue>Приведем значения к нужному нам типу

In [ ]:
df["Год выпуска"] = df["Год выпуска"].astype(int)
df["Пробег"] = df["Пробег"].apply(lambda x: int(str(x)[:-3].replace(u'\xa0',""))) 

df["Владельцы"].fillna(0, inplace=True)
df["Владельцы"] = df["Владельцы"].apply(lambda x: int(str(x)[0]))

<font color=darkblue>Добавим столбец с возрастом авто

In [ ]:
df["Возраст"] = 2018 - df["Год выпуска"]

<font color=darkblue>Добавим столбец "Кузов" (первое слово столбца "Тип кузова")

In [ ]:
df['Кузов'] = df["Тип кузова"].apply(lambda x: x.split()[0])

<font color=darkblue>Удалим ненужные столбцы

In [ ]:
dellist = ["Название", "Стоимость", "Тип двигателя", "Срок владения", "Комментарий", "Тип кузова"]
df = df.drop(dellist, 1)
df.head()

<font color=darkblue>Сохраним полученный датасет.

In [ ]:
df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")

### Диаграмма sankey. КПП - Привод

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import plotly
import plotly.plotly as py

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import random
r = lambda: random.randint(0,255)

In [ ]:
def minimize_data_by_mark(df_in, auto_min_cnt):
    #Отсечем данные по популярным маркам в продаже
    df_temp = df_in.groupby(['Марка']).count().reset_index()
    mark_list = df_temp[(df_temp['Год'] > auto_min_cnt)]['Марка'].tolist()
    df_res = df_in[(df_in['Марка'].isin(mark_list))]
    
    return df_res

<font color=darkblue>Функция для получения датафрейма с данными для plotly sunkey чарта

In [ ]:
def prepare_sankey_data(df_in, attribute_list):
    labels = pd.DataFrame()
    for attr in attribute_list:
        label = pd.DataFrame(df_in[attr].unique(), columns=['label']).reset_index(drop=True)
        labels = pd.DataFrame(pd.concat([labels, label], axis=0))

    labels['color'] = labels['label'].apply(lambda x: ('#%02X%02X%02X' % (r(),r(),r())))
    labels = labels.reset_index(drop=True)
    labels['ID'] = labels.index

    df_res = pd.DataFrame()
    for i, attr in enumerate(attribute_list):
        if (i == len(attribute_list)-1):
            break
        df_tmp = df_in.groupby([attribute_list[i], attribute_list[i+1]]).count().reset_index()
        df_tmp = pd.merge(df_tmp, labels, how='left', left_on=[attr], right_on=['label'])
        df_tmp = pd.merge(df_tmp, labels, how='left', left_on=[attribute_list[i+1]], right_on=['label'])
        df_tmp = df_tmp[['ID_x','ID_y', 'Год']]
        df_tmp.columns = ['source_value', 'dest_value', 'amount']
        df_res = pd.concat([df_res, df_tmp], axis=0)
    
    return labels, df_res

<font color=darkblue>Функция для вормирования справочника для sankey чарта

In [ ]:
def draw_sankey(labels, df_res, title):
    data_trace = dict(
        type='sankey',
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
            pad = 10,
            thickness = 30,
            line = dict(
            color = "black",
            width = 0.5
          ),
        label = labels['label'].tolist(),
        color = labels['color'].tolist(),
        ),
        link = dict(
        source = df_res['source_value'].tolist(),
        target = df_res['dest_value'].tolist(),
        value = df_res['amount'].tolist(),
        ),
    )

    layout =  dict(
        title = title,
        height = 772,
        width = 950,
        font = dict(
            size = 14
        ),    
    )
    
    fig = dict(data=[data_trace], layout=layout)
    iplot(fig, validate=False)

<font color=darkblue>Подготовим датасет
Разобъем пробег по классам

In [ ]:
df['Пробег тип'] = ''
df.loc[df.Пробег < 20000, 'Пробег тип'] = '0-20 тыс.'
df.loc[(df.Пробег >= 20000) & (df.Пробег < 40000), 'Пробег тип'] = '20-40 тыс.'
df.loc[(df.Пробег >= 40000) & (df.Пробег < 60000), 'Пробег тип'] = '40-60 тыс.'
df.loc[(df.Пробег >= 60000) & (df.Пробег < 80000), 'Пробег тип'] = '60-80 тыс.'
df.loc[(df.Пробег >= 80000) & (df.Пробег < 100000), 'Пробег тип'] = '80-100 тыс.'
df.loc[df.Пробег >= 100000, 'Пробег тип'] = 'более 100 тыс.'

<font color=darkblue>Также по количеству владельцев

In [ ]:
df['Количество владельцев'] = df['Владельцы'].apply(lambda x: ("Владельцев: %s" % str(x)))

In [ ]:
#df.loc[df['Просмотров в день'] == 'более 60'] = 60
df['Просмотров в день тип'] = ''
df.loc[df['Просмотров в день'] < 10, 'Просмотров в день тип'] = '0-10 просмотра'
df.loc[(df['Просмотров в день'] >=10) & (df['Просмотров в день'] < 20), 'Просмотров в день тип'] = '10-20 просмотра'
df.loc[(df['Просмотров в день'] >= 20) & (df['Просмотров в день'] < 30), 'Просмотров в день тип'] = '20-30 просмотра'
df.loc[(df['Просмотров в день'] >= 30) & (df['Просмотров в день'] < 40), 'Просмотров в день тип'] = '30-40 просмотра'
df.loc[(df['Просмотров в день'] >= 40) & (df['Просмотров в день'] < 50), 'Просмотров в день тип'] = '40-50 просмотра'
df.loc[(df['Просмотров в день'] >= 50) & (df['Просмотров в день'] < 60), 'Просмотров в день тип'] = '50-60 просмотра'
df.loc[(df['Просмотров в день'] >= 60), 'Просмотров в день тип'] = 'более 60'

In [ ]:
labels, df_res = prepare_sankey_data(df, ['КПП', 'Привод'])

In [ ]:
draw_sankey(labels, df_res, 'Большинство полноприводных машин с автоматической КПП<br>Большинство машин с механической КПП - переднеприводные')

In [ ]:
df_temp = df[(df['Цвет'] == 'красный')]
df_temp = minimize_data_by_mark(df_temp, 20)
labels, df_res = prepare_sankey_data(df_temp, ['Марка', 'Кузов'])
draw_sankey(labels, df_res, 'Красные Nissan в основном внедорожники<br>красные Ford и Renault в основном хэтчбэки<br>красные Mazda и Audi в основном седаны')

In [ ]:
labels, df_res = prepare_sankey_data(df, ['Двигатель', 'Привод'])
draw_sankey(labels, df_res, 'Заднеприводные авто в осноаном с бензиновыми двигателями<br>Дизельные машины в основном полноприводные')

In [ ]:
df_temp = minimize_data_by_mark(df, 800)
df_temp = df_temp[(df_temp['Пробег'] < 100000)]
df_temp = df_temp[(df_temp['Владельцы'] == 1)]
labels, df_res = prepare_sankey_data(df_temp, ['Марка', 'Пробег тип'])
draw_sankey(labels, df_res, 'При каком пробеге владелец (купивший авто с нуля) хочет продать <br> свою машину в зависимости от марки')

<font color=darkblue>Быстрее всего хотят продать Lada, дольше ездить ххотят на Toyota и Volkswagen

In [ ]:
df_temp = minimize_data_by_mark(df, 700)
labels, df_res = prepare_sankey_data(df_temp, ['Марка', 'Просмотров в день тип'])
draw_sankey(labels, df_res, 'Марка - Просмотров в день (интерес к марке)')

<font color=darkblue>Меньше всего просматривают Lada, чаще всего Mercedes и BMW

In [ ]:
df_temp = minimize_data_by_mark(df, 700)
labels, df_res = prepare_sankey_data(df_temp, ['Количество владельцев','Марка','Пробег тип'])
draw_sankey(labels, df_res, 'Марка - Пробег - Количество владельцев')